<a href="https://colab.research.google.com/github/kjh1311990/face/blob/main/Fuzzy_Extractor3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

!pip install komm

Mounted at /content/drive
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for komm: filename=komm-0.7.1-cp36-none-any.whl size=56609 sha256=f3b1ed41843f9d02a8f58083a5df1692bd084e8f7c36b8b8f76447fc1ee1b64d
  Stored in directory: /root/.cache/pip/wheels/22/1e/97/4a65e9a6e202aa360d36ec6a822b31cea491e2ec55788cf71f
Successfully built komm


In [11]:
import komm
import numpy as np
code = komm.BCHCode(10,127)

PARAMS_BCH_N = 1023
PARAMS_BCH_K = 133

In [27]:
import numpy as np

PARAMS_M = PARAMS_BCH_N
PARAMS_N = 128

PARAMS_LOGQ = 8
PARAMS_LOGL = 5
PARAMS_LOGT = 4

PARAMS_Q = 1 << PARAMS_LOGQ
PARAMS_L = 1 << PARAMS_LOGL

PARAMS_TRAINING_SET = 10
PARAMS_FACTOR = 2.5

In [13]:
def EMBLEM_Encode(m, shift):
    encm = ((m << 1) + 1) << (shift - 1);
    return encm.reshape(PARAMS_M, 1);

def EMBLEM_Decode(m, shift):
    decm = m >> shift;
    return decm.reshape(PARAMS_M, 1);

In [14]:
from keras.models import load_model
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import dlib, cv2

detector = dlib.get_frontal_face_detector();
sp = dlib.shape_predictor('/content/drive/MyDrive/Notebooks/shape_predictor_5_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('/content/drive/MyDrive/Notebooks/dlib_face_recognition_resnet_model_v1.dat')


def Setup():
  A = np.random.randint(low = -PARAMS_L/2, high = PARAMS_L/2, size = PARAMS_N * PARAMS_M, dtype = int)
  A.resize(PARAMS_M, PARAMS_N)

  return A;

def Encap(A, folder):
  feature = np.zeros((PARAMS_TRAINING_SET, PARAMS_N));

  for i in range(0,PARAMS_TRAINING_SET):    
    tmp = dlib.load_rgb_image(folder + '/' + str(i) +'.jpg')
    tmp_detection = detector(tmp, 1)
    tmp_shape = sp(tmp, tmp_detection[0])
    tmp_aligned = dlib.get_face_chip(tmp, tmp_shape);
    feature[i] = facerec.compute_face_descriptor(tmp_aligned);
  t = (feature.max(axis=0) - feature.min(axis=0));

  for i in range(0,PARAMS_TRAINING_SET):
    for j in range(0,PARAMS_N):
      feature[i][j] = (feature[i][j] / (PARAMS_FACTOR * t[j]))
  
  b = feature.mean(axis=0).astype(int).reshape(PARAMS_N,1);

  m = np.random.randint(low = 0, high = 2, size = PARAMS_BCH_K, dtype = int);

  encm = code.encode(m);
  encm.reshape(PARAMS_M,1);

  c = np.remainder(np.matmul(A,b) + EMBLEM_Encode(encm, PARAMS_LOGQ-1), PARAMS_Q);

  c = (c + (1 << (PARAMS_LOGQ - PARAMS_LOGT - 1))) >> (PARAMS_LOGQ - PARAMS_LOGT);

  return (t, c, m, b);

def Decap(A, t, c, imgg):
  tmp = dlib.load_rgb_image(imgg)
  tmp_detection = detector(tmp, 1)
  tmp_shape = sp(tmp, tmp_detection[0])
  tmp_aligned = dlib.get_face_chip(tmp, tmp_shape);
  feature = facerec.compute_face_descriptor(tmp_aligned);

  for j in range(0,PARAMS_N):
    feature[j] = (feature[j] / (PARAMS_FACTOR  * t[j]))

  bprime = np.array(feature).reshape(PARAMS_N,1).astype(int);

  c = c << (PARAMS_LOGQ - PARAMS_LOGT);
  cprime = np.remainder(c - np.matmul(A, bprime), PARAMS_Q)
  decm = EMBLEM_Decode(cprime, PARAMS_LOGQ-1).reshape(PARAMS_M)
  mprime = code.decode(decm);
  return (mprime, bprime);

In [28]:
A = Setup();

In [29]:
(t, c, m, b) = Encap(A, '/content/drive/MyDrive/Notebooks/face/input/1');

In [30]:
b.reshape(PARAMS_N)

array([ 0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  3,  0, -1,  0,  0,  3, -1,
       -2,  0,  0,  0,  0,  0,  0, -1, -2, -1,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  1,  0, -1,  0,  0,  3,  1,  0,  0, -1,  0, -1,  0,
        1,  0,  2,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0,  1,  0,  0, -3,
        1, -3, -1,  0, -1, -1, -5,  0,  3,  2, -2,  0,  0,  0,  0,  1,  0,
        0, -1,  0,  3, -1,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,
        0,  0,  0,  1,  0,  0,  0,  0, -2,  0,  0, -2,  3,  1,  0,  0,  1,
        0,  0,  0, -1,  0,  1,  0,  0,  0])

In [31]:
for i in range(0,10):
  (mprime, bprime)= Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/input/1/'+ str(i) + '.jpg');
  print('============================================================================')
  print('comparision ' + str(i));
  print('result :' + str(np.sum(np.abs(m - mprime))));
  print('bprime :' + str(bprime.reshape(PARAMS_N)))
  print('b - bprime :' + str(b.reshape(PARAMS_N) - bprime.reshape(PARAMS_N)))
  #cc = np.remainder(np.matmul(A, bprime),PARAMS_Q) >> (PARAMS_LOGQ - 2);
  #print(np.histogram(cc, bins = np.arange(5)))

comparision 0
result :0
bprime :[ 0  0  0  0  0  0  0 -1  0 -1  3  0 -1 -1  0  3 -1 -2  0  0  0  0  0  0
 -1 -2 -1  0  0  0  0  0 -1  0  0  0  0  0  1  0 -1  0  0  3  1  0  0 -1
  0 -1  0  1  0  1  0  0  1  0 -1  0  0  0  0  0  1  0  0 -3  1 -3 -1  0
 -1 -1 -5  0  3  1 -2  0  0  0  0  1  0  0 -2  0  3 -1  0  4  0  0  0  0
  0  0  0  0  2  0  0  0  0  1  0  0  0  0 -2  0  0 -2  3  1  0  0  1  0
  0  0 -2  0  1  0  0  0]
b - bprime :[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
comparision 1
result :0
bprime :[ 0  0  0  0  0  0  0 -1  0 -1  3  0 -1 -1  0  3 -1 -2  0  0  0  0  0  0
 -1 -2 -1  0  0  0  0  0 -1  0  0  0  0  0  1  0 -1  0  0  3  1  0  0 -1
  0 -1  0  1  0  1  0  0  1  0 -1  0  0  0  0  0  1  0  0 -2  1 -3 -1  0
 -1 -1 -5  0  3  1 -2  0  0  0  0  1  0  0 -1  0  3 -

In [32]:
for i in range(0,5):
  (mprime, bprime)= Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/input/2/'+ str(i) + '.jpg');
  print('============================================================================')
  print('comparision ' + str(i));
  print('result :' + str(np.sum(np.abs(m - mprime))));
  print('bprime :' + str(bprime.reshape(PARAMS_N)))
  print('b - bprime :' + str(b.reshape(PARAMS_N) - bprime.reshape(PARAMS_N)))
  #cc = np.remainder(np.matmul(A, bprime),PARAMS_Q) >> (PARAMS_LOGQ - 2);
  #print(np.histogram(cc, bins = np.arange(5)))

comparision 0
result :0
bprime :[ 0  0  0  0  0  0  0  0  0 -1  3  0 -2  0  0  3 -1 -1  0  0  0  0  0  0
 -1 -1 -1  0  0  0  0  0 -1  0  0  0  0  0  1  0 -1  0  0  3  1  0  0 -1
  0 -1  0  1  0  2  0  0  1  0  0  0  0  0  0  0  1  0  0 -2  1 -3 -1  0
 -1 -1 -5  0  3  1 -1  0  0  0  0  1  0  0 -1  0  3 -1  0  4  0  0  0  0
  0  0  0  0  3  0  0  0  0  1  0  1  0  0 -1  0  0 -2  2  1  0  0  1  1
  0  0 -1  0  0  1  0  0]
b - bprime :[ 0  0  0  0  0  0  0 -1  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0
  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0 -1  0  0  0  0
  0  0  0  0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0 -1  0  0  0  0  0  0 -1  0  0 -1  0  0  0  1  0  0  0  0 -1
  0  0  0  0  1 -1  0  0]
comparision 1
result :0
bprime :[ 0  0  0  0  0  0  0 -1  0 -1  3  0 -2  0  0  3 -1 -1  0  0  0  0  0  0
 -1 -2 -1  0  0  0  0  0 -1  0  0  0  0  0  1  0 -1  0  0  3  1  0  0

In [22]:
for i in range(0,5):
  (mprime, bprime)= Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/input/3/'+ str(i) + '.jpg');
  print('============================================================================')
  print('comparision ' + str(i));
  print('result :' + str(np.sum(np.abs(m - mprime))));
  print('bprime :' + str(bprime.reshape(PARAMS_N)))
  print('b - bprime :' + str(b.reshape(PARAMS_N) - bprime.reshape(PARAMS_N)))
  #cc = np.remainder(np.matmul(A, bprime),PARAMS_Q) >> (PARAMS_LOGQ - 2);
  #print(np.histogram(cc, bins = np.arange(5)))

comparision 0
result :28
bprime :[ 0  0  0 -1  0 -1 -1 -1  1 -2  3  0 -2  0  0  3 -2 -2  0  0  0  0  0  0
 -1 -2 -2  0  0  0  0  0 -1  0  1  1  0  0  1  0 -1  0  1  3  1  0  0 -1
  0 -1  0  1  0  2  1 -1  0  0 -1  0  0  0  0  0  2  0 -1 -4  1 -3 -2  0
 -1 -1 -6  0  3  1 -1  0  0  0  0  2  0  0 -1  0  3  0 -1  2  0  1  0  0
  0  0 -1  0  0  0  0  0  0  0  0  0  0  0 -2  0  1 -3  3  1  0  1  1  0
  0  0 -2  0  0  0  1  0]
b - bprime :[ 0  0  0  1  0  1  1  0 -1  1  0  0  0 -1  0  0  1  0  0  0  0  0  0  0
  0  0  1  0  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0 -1  0  0  0  0  0
  0  0  0  0  0  0 -1  1  1  0  0  0  0  0  0  0 -1  0  1  1  0  0  1  0
  0  0  1  0  0  1 -1  0  0  0  0 -1  0  0 -1  0  0 -1  1  2  0 -1  0  0
  0  0  1  0  2  0  0  0  0  1  0  0  0  0  0  0 -1  1  0  0  0 -1  0  0
  0  0  0  0  1  0 -1  0]
comparision 1
result :26
bprime :[ 0  0  0  0  0  0 -1 -1  1 -1  3  0 -2  0  0  3 -2 -2  0  0  0  0  0  0
 -1 -2 -1  0  0  0  0  0 -1  0  1  1  0  0  1  0 -1  0  1  3  1  0 

In [23]:
for i in range(0,5):
  (mprime, bprime)= Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/input/4/'+ str(i) + '.jpg');
  print('============================================================================')
  print('comparision ' + str(i));
  print('result :' + str(np.sum(np.abs(m - mprime))));
  print('bprime :' + str(bprime.reshape(PARAMS_N)))
  print('b - bprime :' + str(b.reshape(PARAMS_N) - bprime.reshape(PARAMS_N)))
  #cc = np.remainder(np.matmul(A, bprime),PARAMS_Q) >> (PARAMS_LOGQ - 2);
  #print(np.histogram(cc, bins = np.arange(5)))

comparision 0
result :27
bprime :[ 0  0  0  0 -1  0  0 -1  0 -1  4  0 -1 -1  0  4 -2 -2  0  0  0  0  0  0
 -1 -2 -1  0  0  0 -1  0 -1  0  0  1  0 -1  0  0 -1  0  1  3  1  0  0 -1
  0 -1  0  0  0  1  0 -1  0  0  0  0  0  0  0 -1  2  0  0 -3  1 -1  0  0
 -1 -1 -5  0  3  1 -1  0  0  0  0  1  0  0 -1  0  2  0  0  3  0  0  0  0
  0  2  0  0  2  0  0  0  0  0  0  0  0  0 -2  0  0 -4  3  2  0  0  1  0
  0  0 -1  0  0 -1  0  0]
b - bprime :[ 0  0  0  0  1  0  0  0  0  0 -1  0 -1  0  0 -1  1  0  0  0  0  0  0  0
  0  0  0  0  0  0  1  0  0  0  0 -1  0  1  1  0  0  0 -1  0  0  0  0  0
  0  0  0  1  0  1  0  1  1  0 -1  0  0  0  0  1 -1  0  0  0  0 -2 -1  0
  0  0  0  0  0  1 -1  0  0  0  0  0  0  0 -1  0  1 -1  0  1  0  0  0  0
  0 -2  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  2  0 -1  0  0  0  0
  0  0 -1  0  1  1  0  0]
comparision 1
result :26
bprime :[ 0  0  0  0 -1  0  0 -1  0 -1  3  0 -1 -1  0  4 -2 -2  0  0  0  0  0  0
 -1 -2 -1  0  0  0 -1  0 -1  0  0  1  0 -1  0  0 -1  0  0  2  1  0 

In [24]:
for i in range(0,5):
  (mprime, bprime)= Decap(A, t, c, '/content/drive/MyDrive/Notebooks/face/input/5/'+ str(i) + '.jpg');
  print('============================================================================')
  print('comparision ' + str(i));
  print('result :' + str(np.sum(np.abs(m - mprime))));
  print('bprime :' + str(bprime.reshape(PARAMS_N)))
  print('b - bprime :' + str(b.reshape(PARAMS_N) - bprime.reshape(PARAMS_N)))
  #cc = np.remainder(np.matmul(A, bprime),PARAMS_Q) >> (PARAMS_LOGQ - 2);
  #print(np.histogram(cc, bins = np.arange(5)))

comparision 0
result :22
bprime :[ 0  0  0  0  0  0  0 -1  1 -1  3  0 -2  0  0  2 -1 -1 -1  0  0  0  0  0
 -1 -2  0  0  0  0  0  0 -1  0  0  1  0 -1  1  0 -1  0  0  3  0  0  0 -1
  0 -1  0  1  0  2  0 -1  1  0  0  0  0 -1  0  0  1  0  0 -3  0 -3 -1  0
 -1 -1 -5  0  3  2 -1  0  0  0  1  1  0  0 -1  0  3  0 -2  3  0  1  0  0
  0  0 -1  0  1  0  0  0  0  0  0  0  0  0 -1  0  0 -3  2  1  1  1  1  0
  0  0 -2  0  1  2  1  0]
b - bprime :[ 0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  1  0 -1  1  0  0  0  0  0
  0  0 -1  0  0  0  0  0  0  0  0 -1  0  1  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0  0  1  0  0 -1  0  0  1  0  0  0  0  0  0  1  0  0  0
  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0 -1  0  0 -1  2  1  0 -1  0  0
  0  0  1  0  1  0  0  0  0  1  0  0  0  0 -1  0  0  1  1  0 -1 -1  0  0
  0  0  0  0  0 -2 -1  0]
comparision 1
result :26
bprime :[ 0  0  0  0  0  0  0 -1  1 -1  3  0 -2  0  0  2 -1 -1 -1  0  0  0  0  0
 -1 -2  0  0  0  0  0  0 -1  0  1  1  0  0  1  0 -1  0  0  3  0  0 

In [25]:
cc = np.remainder(np.matmul(A, b),PARAMS_Q) >> (PARAMS_LOGQ - 2);

In [26]:
np.histogram(cc, bins = np.arange(5))

(array([247, 264, 250, 262]), array([0, 1, 2, 3, 4]))